![image source: Amazon Nova Canvas. (Prompt: Giant mushroom, stanley artgerm, pointillism, bokeh, sparkles, magic, trees, mountains, lake, thomas kinkaid, norman rockwell)](images/2025-01-26-streamlit-bedrock-nova-app.jpg)

## Introduction

[Amazon's Nova Canvas](https://aws.amazon.com/ai/generative-ai/nova/creative/) is a state-of-the-art Amazon foundation model that promises to democratize high-quality image generation. However, the complexities of these models can often be a barrier to entry. This post presents a practical solution: a [Streamlit-based](https://streamlit.io/) application that makes experimenting with Nova Canvas accessible to everyone. I will walk you through the features of this app, which includes five powerful tools, encompassing text-to-image generation, conditional generation, background removal, inpainting, and outpainting. You can seamlessly explore the diverse capabilities of this potent technology and bring your creative visions to life.

### Environment Details

In [1]:
#| code-fold: true
#| code-summary: "Show the code"

from platform import python_version
import streamlit

print("python==" + python_version())
print("streamlit==" + streamlit.__version__)

python==3.12.8
streamlit==1.41.1


::: {.callout-tip}
#### Code Samples

All the code examples used in this post can be found on the GitHub repo [2025-01-26-streamlit-bedrock-nova-app](https://github.com/hassaanbinaslam/2025-01-26-streamlit-bedrock-nova-app)

:::

## Introducing Amazon Nova Canvas

At the heart of my app is Amazon Nova Canvas, a latest foundation model that I'm using to generate realistic, studio-quality images from text prompts. This model is a versatile tool for both text-to-image generation and advanced image editing. Nova Canvas excels at creating images that capture the concepts described in your text prompts.

Beyond `text-to-image` (T2I), I'm also leveraging its image editing capabilities, which include `inpainting`, `outpainting`, generating variations, and automatic editing without masks. **Inpainting** allows you to reconstruct masked regions in an image. **Outpainting** lets you seamlessly extend an image beyond its original boundaries, and you can guide this with masks or let the model determine it. The model also supports **image conditioning**, which uses a reference image to guide generation, and it also supports subject consistency to preserve a chosen subject like a pet or a shoe in your generated images. It can also automatically `remove backgrounds` from an image, making the background transparent.

Overall, this model provides me with a powerful and diverse set of tools to unleash creative possibilities.

::: {.callout-tip}
#### Amazon Nova Canvas Official Documentation

The Nova Canvas model id used in this post is `amazon.nova-canvas-v1:0`.

You can read more about Nova Canvas features from its official documentation page. [Link Here](https://docs.aws.amazon.com/nova/latest/userguide/image-generation.html)

:::

## The Rapid Prototyping App: A Hands-On Playground

To make exploring the power of Amazon Nova Canvas as easy and accessible as possible, I decided to build an application using [Streamlit](https://streamlit.io/). I chose Streamlit because it is incredibly efficient for quickly creating interactive web apps with minimal coding, making it the perfect tool for this project. My goal was to create a hands-on playground where you could directly experiment with Nova Canvas's capabilities, transforming your creative ideas into visual reality.

**This app isn't just for tech enthusiasts; I wanted it to be user-friendly, allowing anyone to explore AI-powered image generation and editing**. The best part is that it's designed for easy sharing and deployment, so you can effortlessly share your experiments with your friends, or even deploy the app for wider access. I have included some of the app's screenshots below, so you can have a quick overview.

You'll see a simple login page where you can enter your credentials (or bypass authentication during local testing). After logging in, you are presented with a user-friendly interface, with a sidebar that lets you navigate to each of the five image processing tools. The welcome page gives you a quick overview of each of the tools and how to use them. Each tool provides an intuitive interface with controls to configure the parameters and get the most out of the capabilities of the Amazon Nova Canvas model.

::: {layout-ncol=3}
![Login Page](images/2025-01-26-streamlit-bedrock-nova-app/login.png){.lightbox}

![Welcome Page](images/2025-01-26-streamlit-bedrock-nova-app/welcome.png){.lightbox}

![Text to Image Page](images/2025-01-26-streamlit-bedrock-nova-app/text-2-image.png){.lightbox}
:::

::: {layout-ncol=3}
![Text to Image with Condition Page](images/2025-01-26-streamlit-bedrock-nova-app/image-conditioning.png){.lightbox}

![Remove Image Background Page](images/2025-01-26-streamlit-bedrock-nova-app/image-conditioning.png){.lightbox}

![Image Inpainting Page](images/2025-01-26-streamlit-bedrock-nova-app/image-inpainting.png){.lightbox}
:::

::: {layout-ncol=3}

![Image Outpainting Page](images/2025-01-26-streamlit-bedrock-nova-app/image-outpainting.png){.lightbox}

:::

We'll dive into each of these pages in more detail in the following sections, but this overview should give you a good initial feel of what the app looks and feels like.

## Text to Image

The first tool in our arsenal is the "Text to Image" generator. This is where the magic begins: It let's you turn your imagination into stunning visuals with just a few words. By harnessing the advanced capabilities of Nova Canvas, you can generate high-quality images tailored to your specific prompts.

#### How to use it:

* Use the controls on the page to configure the text-to-image generation process. You can select the number of images you want, the size of each image (with a variety of preset options), and fine-tune the cfgScale parameter.
* The `cfgScale` parameter lets you specify how strongly the generated image should adhere to the prompt. A lower value introduces more randomness, while a higher value will stick more closely to the prompt.
* Enter a positive prompt – this is where you describe the image you want to generate. Be as specific as possible to get the best results.
* Optionally, you can also enter a negative prompt. This is a way to tell the model what you don't want to see in the image, further refining the output.

Once you are happy with the parameters you have selected, simply click the "Generate Image" button to start the process. The generated images will be displayed below the button.

Below is an example of what the app looks like after generating two images. I used the prompt **Fantastic celestial night sky with shining stars. Amazing John Dyer art.** using the following settings.

![](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/txt2img-with-results.png){.lightbox}

I encourage you to experiment with different prompts and parameters to see the amazing results that you can create using this tool. I have provided below more examples of the generated images with different prompts to demonstrate the versatility of the tool.

::: {.callout-tip}
Click on an image to expand it, and view in a higher resolution.
:::

::: {layout-ncol=4}
![A serene forest where the trees are made of glass, sunlight refracting into rainbow colors, with a mystical glowing lake in the center. Style: fantasy realism.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-1.jpg){.lightbox}

![A giant clock tower covered in vines, standing in the middle of a desert, with time flowing like water around it. Style: surreal art, soft light](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-2.jpg){.lightbox}

![A massive volcanic mountain surrounded by a swirling galaxy in the night sky, with rivers of lava glowing in the dark. Style: cinematic and dramatic.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-3.jpg){.lightbox}

![An underwater city glowing with bioluminescent plants and creatures, built into a coral reef, with massive whales swimming above. Style: fantasy underwater.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-4.jpg){.lightbox}
:::

::: {layout-ncol=4}
![A futuristic Mars colony with advanced architecture, glowing biodomes, and red dunes stretching to the horizon. Style: sci-fi realism.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-5.jpg){.lightbox}

![A warrior elf with glowing silver armor, holding a sword that emits blue flames, standing in an enchanted forest. Style: hyper-detailed fantasy art.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-6.jpg){.lightbox}

![A steampunk inventor with mechanical goggles, surrounded by intricate clockwork machines, in a smoky Victorian-era workshop. Style: steampunk, warm tones.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-7.jpg){.lightbox}

![A robotic wolf with glowing red eyes, prowling through a futuristic cityscape at night. Style: cyberpunk, metallic textures.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-8.jpg){.lightbox}
:::

::: {layout-ncol=4}
![A giant turtle with a miniature forest growing on its back, walking through the ocean at sunrise. Style: serene, magical realism.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-9.jpg){.lightbox}

![A massive spaceship docked on a floating platform above a gas giant, with bright auroras in the sky. Style: sci-fi cinematic.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-10.jpg){.lightbox}

![A futuristic AI core glowing in the center of a high-tech laboratory, surrounded by floating holograms and robotic arms. Style: sleek and ultra-modern.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-11.jpg){.lightbox}

![An explosion of colors forming a human face, with geometric patterns and abstract swirls blending into the background. Style: modern art.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-12.jpg){.lightbox}
:::

::: {layout-ncol=4}
![A dream-like landscape where the sky is made of water and the ground is a glowing galaxy, with floating islands in between. Style: surreal fantasy.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-13.jpg){.lightbox}

![A haunted Victorian mansion on a cliffside, illuminated by lightning strikes, with ghostly figures in the windows. Style: gothic horror.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-14.jpg){.lightbox}

![A dark angel with tattered black wings, standing in a crumbling cathedral, with glowing red eyes and a flaming sword. Style: dark fantasy.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-15.jpg){.lightbox}

![A cursed forest with twisted trees, glowing red mushrooms, and a shadowy figure lurking in the fog. Style: eerie and atmospheric.](images/2025-01-26-streamlit-bedrock-nova-app/txt2img/prompt-16.jpg){.lightbox}
:::

## Text to Image with Condition

Taking image generation to the next level, my "Text to Image with Condition" tool lets you shape your creations with precision and intention. It goes beyond simple text prompts by using a reference image to guide the generation process, helping you achieve the exact look and feel you desire. This tool uses advanced image conditioning features, offering two powerful methods: Canny Edge Detection and Segmentation Maps.

#### Features:

* **Canny Edge Detection**: By extracting the edges from your reference image. The output will inherit the structural layout of the reference image.
* **Segmentation Maps**: This allows you to define and control specific areas or objects in the reference image. The model will then use this information to render precise outputs and allow you to manipulate individual objects in the image.

#### How to Use It:

* First, upload the reference image that you want to use to guide the generation. For best results, ensure the reference image is clear and appropriately sized.
* Next, select the conditioning method that you want to use: Canny Edge or Segmentation Map from the "Control Mode" dropdown.
* You can adjust the "Control Strength" parameter to determine how closely the generated image adheres to the reference image. Use a lower value to introduce more randomness in the generation.
* Then, enter a text prompt describing the image you want to generate, keeping in mind the selected conditioning method.
* Optionally, you can also enter a negative prompt to further refine the image.
* Configure other options, such as the number of images, image size and the cfgScale parameter, just like you did in the Text to Image page.
* Click on the "Generate Image" button to view the results. The generated images will be displayed below the button along with the original image used for conditioning.

Below is an example of what the app looks like after generating two images using the Canny Edge option. I used the reference image (Harry Potter/Daniel Radcliffe) shown in the "Uploaded Image" section and the prompt **Reimagine this person in a cyberpunk cityscape with glowing neon lights.** The other options were set to the default value.

![](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/image-condition-result.png){.lightbox}

This tool allows you to take full control of the image generation process and unleash your creative possibilities. I have shared below some more examples of the generated images with different prompts, and the input image used for the condition.

::: {layout-ncol=2}
![Input Condition Image](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/Dumbledore.jpg){.lightbox}

![Transform this wizard into a celestial guardian with a starry cloak, glowing golden staff, and galaxies swirling in his beard. Style: ethereal fantasy.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-1.jpg){.lightbox}
:::

Some more examples.

::: {layout-ncol=4}
![Input Condition Image](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/Dumbledore.jpg){.lightbox}

![Transform him into a robotic AI wizard with a metallic body, LED lights in his eyes, and a holographic spellbook floating in front of him.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-2.jpg){.lightbox}

![Turn him into a dark necromancer with a tattered black cloak, glowing green skulls floating around him, and a staff topped with a glowing crystal skull.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-3.jpg){.lightbox}

![Make him an ice wizard with a robe made of frost and snowflakes, a frozen staff, and icy blue magic swirling around his hands.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-4.jpg){.lightbox}
:::

::: {layout-ncol=4}
![Input Condition Image](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/forrest-scene.png){.lightbox}

![Enhance this forest with floating lanterns and mystical creatres like fairies and glowing butterflies.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-5.jpg){.lightbox}

![Turn this forest into a futuristic biome with glass domes and flying drones.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-6.jpg){.lightbox}

![Add a hidden magical portal surrounded by glowing mushrooms and ancient runes.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-7.jpg){.lightbox}
:::

::: {layout-ncol=4}
![Input Condition Image](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/cat.jpg){.lightbox}

![Turn this cat into a futuristic robot with glowing eyes and metal plating.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-8.jpg){.lightbox}

![Make this cat appear as a royal king with a golden crown and cape.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-9.jpg){.lightbox}

![Reimagine this cat as a mythical fire spirit surrounded by flames.](images/2025-01-26-streamlit-bedrock-nova-app/img-condition/prompt-10.jpg){.lightbox}
:::

## Background Removal

Sometimes, you need a clean subject without any background distractions. That's where my "Background Removal Tool" comes in. This tool empowers your creative workflow by allowing you to effortlessly remove backgrounds from images in just one step. Whether you want to composite your subject onto a solid color background or seamlessly layer it over another scene, this tool provides a clean and accurate solution.

Leveraging the power of Nova Canvas, the app intelligently detects and segments multiple foreground objects, ensuring even complex scenes with overlapping elements are isolated with precision.

#### How to Use It:

* Upload an image (PNG or JPG) using the uploader on the page. The app can handle different image formats for your convenience.
* Once uploaded, the app automatically processes the image using Amazon Bedrock to remove the background.
* The processed image, with the background removed, will then be displayed below the uploader.

It's that simple! With this tool, you can transform your images with ease. I'll share some examples of the before and after results of the processed images.

![](images/2025-01-26-streamlit-bedrock-nova-app/background-remove/remove-img-backgroud-results.png){.lightbox}

Some more examples.

::: {layout-ncol=2}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/background-remove/parrot.jpg){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/background-remove/parrot-removed.png){.lightbox}
:::

::: {layout-ncol=2}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/background-remove/cybertruck.png){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/background-remove/cybertruck-removed.png){.lightbox}
:::

::: {layout-ncol=2}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/background-remove/speaker.png){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/background-remove/speaker-removed.png){.lightbox}
:::

## Image Inpainting

Sometimes, images have unwanted elements or imperfections. That's when my "Image Inpainting Tool" becomes invaluable. This tool allows you to enhance, restore, or creatively modify images by masking specific areas and providing detailed prompts to guide the process. Whether you're fixing imperfections or adding new elements, this tool empowers you with advanced inpainting features.

#### How It Works:

* **Upload an Image**: Begin by uploading the image you wish to modify using the uploader. The app supports various image formats for your convenience.
* **Mask the Area**: Use the integrated canvas tool to intuitively draw masks over the regions you want to change. You can adjust the "Canvas Stroke Width" to fine-tune the size of the brush for precise masking.
* **Provide a Prompt**: Describe the changes you'd like to see (e.g., "replace the masked area with a sunset sky"). Be as specific as possible to guide the inpainting.
* **Generate Results**: Once you are happy with the mask and the prompt, simply click the "Edit Image" button to start the process. The generated output will be displayed below along with the original image and the masked image.

Below is an example of how to use this tool. I have uploaded an image and have masked a section of the image, and using the prompt "Replace the masked area with a honey bee" I was able to generate the image displayed below.

![](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/inpainting-results.png){.lightbox}

This tool gives you a high degree of control over the modifications, allowing you to achieve seamless and natural-looking results. I'll share below some more examples of how you can creatively use this tool.

::: {layout-ncol=3}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/iguana.jpg){.lightbox}

![Masked Image with Prompt: Replace the masked area with a honey bee.](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/iguana-masked.png){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/iguana-output.jpg){.lightbox}
:::

::: {layout-ncol=3}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/cereal-cat.jpg){.lightbox}

![Masked Image with Prompt: Replace the masked area with a dog.](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/cat-masked.png){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/cereal-dog.jpg){.lightbox}
:::

::: {layout-ncol=3}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/sunset-plane.jpg){.lightbox}

![Masked Image with Prompt: Remove from the scene.](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/sunset-mask.png){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/sunset-plane-removed.jpg){.lightbox}
:::

::: {layout-ncol=3}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/tea-spilled.jpg){.lightbox}

![Masked Image with Prompt: Remove the tea stains from the masked area. Table cloth should be clean and tidy.](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/tea-mask.png){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/img-inpainting/tea-stain-removed.jpg){.lightbox}
:::

## Image Outpainting

My final tool in this application is the "Image Outpainting Tool." This tool lets you expand your images beyond their original boundaries, creating seamless extensions that match the original content. Whether you want to extend landscapes, complete partial scenes, or create panoramic views, this tool helps you achieve natural-looking results.

#### How It Works:

* **Upload Image**: Start by uploading your image using the uploader. The app can handle different image formats.
* **Select Mask Type**:
  * **Prompt**: Use text to define the mask area to preserve. You can use a text prompt to specify what part of the image you want to keep unchanged and the model will generate a mask for you.
  * **Image**: Automatically mask the image area to keep unchanged. The model will mask the entire image as the area to preserve.
* **Set Expanded Size**: Choose the new image size: `512x512` or `1024x1024` using the dropdown.
* **Adjust Position**: Use the "Image Horizontal Position" and "Image Vertical Position" controls to place your image within the expanded canvas.
* **Add a Prompt**: Describe how the extended areas should look. Be as specific as possible to get desired results.
* **Generate Results**: Click on the "Edit Image" button to create and refine your expanded image. The expanded image and the expanded image mask will be displayed below the image and controls. The output will be displayed below that.

Below is an example of what the app looks like after outpainting an uploaded image. Here I have used the "Image" mask type and the prompt "forest setting in the background with animals and plants." I have kept the other options at their default value.

![](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/img_outpainting-results.png){.lightbox}

With this tool, you can easily bring your creative visions to life by expanding images beyond their boundaries!I have provided below more examples of the creative possibilities of this tool.


::: {layout-ncol=3}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/panda.jpg){.lightbox}

![Expanded Image. Prompt: forest setting in the background with animals and plants. Mask Prompt: a panda](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/panda_expanded.jpg){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/panda-output.jpg){.lightbox}
:::

::: {layout-ncol=3}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/cat.jpg){.lightbox}

![Expanded Image. Prompt: The cat is surrounded by toys. Mask Prompt: a cat](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/cat-expanded.jpg){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/cat-output.jpg){.lightbox}
:::

::: {layout-ncol=3}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/chair.jpg){.lightbox}

![Expanded Image. Prompt: The chair is placed in a luxury home. Mask Prompt: a chair](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/chair-expanded.jpg){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/chair-output.jpg){.lightbox}
:::

::: {layout-ncol=3}
![Input Image](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/car.jpg){.lightbox}

![Expanded Image. Prompt: The car is sitting on the road with trees, houses, shops. and people walking in the background. Mask Prompt: a car](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/car-expanded.jpg){.lightbox}

![Output Image](images/2025-01-26-streamlit-bedrock-nova-app/img-outpainting/car-output.jpg){.lightbox}
:::

## Running the App Locally

Before deploying the app, you might want to experiment with it on your local system. Here's how you can quickly get the app up and running:

### Prerequisites

* **Amazon Bedrock Access**: You'll need access to the Amazon Nova Canvas model. If you don't have access already, visit the Model Catalog in Amazon Bedrock from your AWS account console and request access to the model (as shown below).
  * ![](images/2025-01-26-streamlit-bedrock-nova-app/nova-canvas-access.png){.lightbox}
* **AWS CLI Configuration**: Make sure you have the AWS CLI configured on your local system and that the user has the necessary permissions to invoke the Nova Canvas model.
  
```{.yaml filename="IAM Policy"}
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": "*"
        }
    ]
}
```

### Code

* **Code Download**: Download the code from the GitHub repository: [2025-01-26-streamlit-bedrock-nova-app](https://github.com/hassaanbinaslam/2025-01-26-streamlit-bedrock-nova-app).
* **Code Structure**: The code is organized as follows:

```{filename="Code Hierarchy"}
├── home
│   └── welcome.py
├── llm
│   ├── __init__.py
│   └── utils.py
├── tools
│   ├── inpainting.py
│   ├── outpainting.py
│   ├── remove_background.py
│   ├── text_to_image_condition.py
│   ├── text_to_image.py
│   └── __init__.py
├── .gitignore
├── app.py
├── config.py
├── Dockerfile
├── README.md
└── requirements.txt
```

Here's a brief explanation of the important files:

**config.py**: This file is used to set up important configurations for the app. Below is the snippet from the file to give you an overview.

```{.python filename="config.py"}
class Config:
    # The name of the Bedrock model to use
    BEDROCK_MODEL = {
        "id": "amazon.nova-canvas-v1:0",
        "name": "Amazon Bedrock - Nova Canvas",
    }

    # If Bedrock is not activated in us-east-1 in your account, set this value accordingly
    BEDROCK_REGION = "us-east-1"

    # User Auth?
    LOGIN_REQUIRED = False

```

In this file, you can specify the Bedrock model ID and region. Make sure that the `BEDROCK_REGION` matches the region where you have access to the Nova Canvas model. You can also disable Cognito user authentication for local testing by setting `LOGIN_REQUIRED` to False.

**requirements.txt**: This file lists the required Python libraries for the app. Make sure these dependencies are installed in your local environment.

```{.txt filename="requirements.txt"}
streamlit==1.41.1
streamlit-drawable-canvas==0.8.0
streamlit-cognito-auth==1.3.1
pillow==11.1.0
boto3==1.35.97
numpy==2.2.1
```

### Running the App

* **app.py**: This is the main file to run the Streamlit application.
* Open your terminal or command prompt.
* Navigate to the directory where you have downloaded the code.
* Run the command `streamlit run app.py`. You should see the following output in your terminal:

![](images/2025-01-26-streamlit-bedrock-nova-app/local-run-app.png){.lightbox}

Open your web browser and visit the URL shown in the output (http://localhost:8501) to access the app.

## Deployment

Now that you've experimented with the app locally, it's time to deploy it to the cloud. I'll be focusing on using [Amazon Lightsail](https://docs.aws.amazon.com/lightsail/latest/userguide/what-is-amazon-lightsail.html), which is a cost-effective and easy-to-use service for deploying containerized applications.

::: {.callout-note title="Alternatives to Lightsail?" collapse="false"}

If you are interested in learning about the alternative approaches in deploying this app, then

- Refer to my other post where I have explained some of the alternative: [2025-01-21-streamlit-deployment-aws](https://hassaanbinaslam.github.io/myblog/posts/2025-01-21-streamlit-deployment-aws.html)
- Refer to the AWS blog post for deploying the Streamlit app with AWS CDK: [build-and-deploy-a-ui-for-your-generative-ai-applications-with-aws-and-python](https://aws.amazon.com/blogs/machine-learning/build-and-deploy-a-ui-for-your-generative-ai-applications-with-aws-and-python/)

:::

Here's a step-by-step guide for deploying your Streamlit app on Amazon Lightsail:

### 1. Create an Elastic Container Registry Repository

Head over to your AWS console and create a new [Elastic Container Registry (ECR)](https://docs.aws.amazon.com/AmazonECR/latest/userguide/what-is-ecr.html) repository. This repository will be used to store your application's Docker container image. I would recommend that you use a descriptive name for your repo, such as `streamlit/bedrock-nova-app`.

Make sure you create the ECR in the same AWS region where you have access to the Amazon Bedrock models.

::: {.callout-note title="Can I use a public repo like Docker Hub?" collapse="false"}

I have used AWS private ECR repo in the above step. But you are open to use any other public service of your choice like [Docker Hub](https://hub.docker.com/).

:::

![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/ecr-repo.png){.lightbox}

### 2. Build and Push the Docker Image

On your local machine, build the Docker container image and then push it to your newly created ECR. Use the commands provided by AWS for your specific account and registry. Here is an example of the commands, specific to my account. Remember that the command provided by AWS in the console is very specific to your account and you should use that only:

![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/ecr-push-commands.png){.lightbox}

### 3. Create a Lightsail Container Service

Navigate to the AWS Lightsail service in your console. We'll be using [Lightsail container service](https://docs.aws.amazon.com/lightsail/latest/userguide/amazon-lightsail-container-services.html).

Lightsail's interface is very intuitive. Follow the guided steps to create the container service, making sure you select the same region where your ECR repository is located. For a simple Streamlit app, a Nano compute power `(512MB RAM, 0.25 vCPU)` will be sufficient.

### 4. Give Lightsail Access to ECR

In the "Images" tab of your Lightsail container service, select the correct ECR repository and grant Lightsail access to it. This allows Lightsail to pull the container image from your repository.

![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/lightsail-ecr-access.png){.lightbox}

::: {.callout-tip title="Containers on AWS — Amazon Lightsail Containers" collapse="false"}

Here is a link to other post that explains the similar step in a very detailed manner: [containers-on-aws-amazon-lightsail](https://dev.to/aws-builders/containers-on-aws-amazon-lightsail-containers-4eco)

:::

### 5. Configure IAM User Permissions

To enable your Lightsail container to access Bedrock models, you'll need to create a new IAM user.

* Assign the `bedrock:InvokeModel` policy to this user.
* Create an access key for the user and save the `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` for later use.

Amazon Lightsail is a special service and you cannot assign IAM roles/permissions to it like you do for other services. Similarly Lightsail cannot access [AWS Secrets Manager](https://docs.aws.amazon.com/secretsmanager/latest/userguide/intro.html) to keep the credentails. So we will create a new IAM user with the required persmissions, and assign it to our app.

![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/iam-app-user.png){.lightbox}

::: {.callout-caution title="Keep your IAM access and secret key secure!" collapse="false" }

Do not share your AWS access key ID or secret access key publicly, and keep them secure.

:::

### 6. Configure Cognito User Pool (If Needed)

If you have enabled user authentication (`LOGIN_REQUIRED = True`) in your `config.py` file, you'll need to set up a Cognito User Pool.

Create a Cognito User Pool and get the `POOL_ID`, `APP_CLIENT_ID`, and `APP_CLIENT_SECRET`.

Note: Do not share your app client secret publicly and keep it secure!

::: {layout-ncol=3}
![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/cognito-user-pool.png){.lightbox}

![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/cognito-pool-id.png){.lightbox}

![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/cognito-client-config.png){.lightbox}
:::

### 7. Deploy the Container

In the "Deployments" tab of your Lightsail container service, provide the following environment variables:

* AWS_ACCESS_KEY_ID
* AWS_SECRET_ACCESS_KEY

If Cognito is enabled

* POOL_ID
* APP_CLIENT_ID
* APP_CLIENT_SECRET

Set the container port to 8501 and the protocol to HTTP.

Click the "Deploy" button.

![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/lightsail-deployment.png){.lightbox}

Your app should now be deployed and accessible via the public domain URL provided by Lightsail! For login you will be required to create a `verified cognito user`.

::: {layout-ncol=2}
![](images/2025-01-26-streamlit-bedrock-nova-app/deployment/cognito-user.png){.lightbox}

![](images/2025-01-26-streamlit-bedrock-nova-app/login.png){.lightbox}
:::

## Conclusion

In this post, I've shared my journey of creating an application using Streamlit and the powerful Amazon Nova Canvas model. From generating images from text to performing advanced editing operations like inpainting and outpainting, this app offers a hands-on experience with some of the latest advancements in AI-powered image processing.

I've shown you how to get started by running the app locally, and I've also provided a detailed guide on deploying the application to AWS using Lightsail. Now, you can easily create amazing visuals using the different tools that I have developed and even share it with your friends.

I encourage you to explore the app, experiment with different prompts and parameters, and unleash your creativity. This is a starting point, and I'm excited to see what you create with these tools. I’d also love to hear your feedback and any ideas you have for future enhancements!

Thank you for joining me on this journey. I hope this app inspires your creative projects and helps you discover the exciting possibilities of AI-powered image generation.